In [1]:
##Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import joblib

In [2]:
# 1. Chargement des données 
data=pd.read_csv("../data/donnees_dpe_rhone_clean.csv",delimiter=",")

In [3]:
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
data.head(3)

,configuration_installation_chauffage_n2,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,numero_voie_ban,score_ban,conso_chauffage_generateur_n1_installation_n2,conso_auxiliaires_ep,conso_chauffage_installation_chauffage_n2,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_ecs_ef_energie_n3,conso_5_usages_par_m2_ep,date_etablissement_dpe,conso_ecs_ef_energie_n1,conso_ecs_ef_energie_n2,emission_ges_chauffage,description_installation_chauffage_n2,description_installation_chauffage_n1,qualite_isolation_plancher_haut_comble_amenage,conso_5_usages_par_m2_ef,cout_ecs_energie_n2,conso_chauffage_ef_energie_n1,cout_ecs_energie_n3,conso_chauffage_ef_energie_n2,presence_brasseur_air,qualite_isolation_menuiseries,conso_chauffage_ef_energie_n3,cout_total_5_usages_energie_n2,cout_total_5_usages_energie_n3,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,isolation_toiture,modele_dpe,qualite_isolation_enveloppe,conso_chauffage_generateur_n1_installation_n1,type_energie_n1,emission_ges_eclairage,type_energie_n2,type_energie_n3,code_postal_ban,conso_5_usages_ef_energie_n3,emission_ges_ecs,conso_5_usages_ef_energie_n2,conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_insee_ban,deperditions_planchers_bas,conso_5_usages_ep,date_fin_validite_dpe,deperditions_enveloppe,complement_adresse_batiment,code_region_ban,volume_stockage_generateur_n1_ecs_n1,surface_chauffee_installation_chauffage_n2,surface_chauffee_installation_chauffage_n1,version_dpe,besoin_ecs,coordonnee_cartographique_x_ban,type_generateur_chauffage_principal,type_energie_principale_ecs,apport_solaire_saison_chauffe,adresse_ban,deperditions_renouvellement_air,_rand,surface_habitable_desservie_par_installation_ecs_n1,production_electricite_pv_kwhep_par_an,nombre_niveau_logement,surface_habitable_logement,cout_ecs,type_installation_ecs_n1,emission_ges_5_usages_energie_n1,emission_ges_5_usages_energie_n2,apport_interne_saison_froide,emission_ges_5_usages_par_m2,description_generateur_chauffage_n1_installation_n1,apport_interne_saison_chauffe,apport_solaire_saison_froide,emission_ges_5_usages_energie_n3,type_generateur_n1_installation_n1,nombre_logements_desservis_par_installation_ecs_n1,type_generateur_n1_installation_n2,cout_auxiliaires,type_emetteur_installation_chauffage_n2,type_emetteur_installation_chauffage_n1,besoin_chauffage,configuration_installation_ecs_n1,description_installation_ecs_n1,description_generateur_chauffage_n1_installation_n2,classe_inertie_batiment,deperditions_ponts_thermiques,type_generateur_chauffage_principal_ecs,emission_ges_refroidissement,hauteur_sous_plafond,conso_chauffage_ef,nom_commune_ban,_geopoint,date_visite_diagnostiqueur,type_batiment,periode_construction,conso_ecs_ep,protection_solaire_exterieure,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,deperditions_baies_vitrees,type_energie_generateur_n1_ecs_n1,ubat_w_par_m2_k,numero_etage_appartement,nom_commune_brut,conso_ef_installation_ecs_n1,etiquette_dpe,description_generateur_n1_ecs_n1,code_departement_ban,type_installation_chauffage_n1,type_installation_chauffage_n2,methode_application_dpe,adresse_brut,logement_traversant,cout_total_5_usages,categorie_enr,conso_refroidissement_ef,conso_eclairage_ef,deperditions_planchers_hauts,zone_climatique,conso_ef_generateur_n1_ecs_n1,emission_ges_ecs_energie_n1,emission_ges_ecs_energie_n2,emission_ges_ecs_energie_n3,cout_refroidissement,conso_chauffage_ep,usage_generateur_n1_installation_n2,conso_eclairage_ep,usage_generateur_n1_installation_n1,nom_rue_ban,qualite_isolation_murs,type_installation_solaire_n1,classe_altitude,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,emission_ges_chauffage_energie_n3,besoin_refroidissement,emission_ges_chauffage_energie_n2,indicateur_confort_ete,emission_ges_chauffage_ene

In [4]:
# Supposons que la colonne cible est "Etiquette_DPE" et qu'il y a des colonnes catégorielles et numériques
target = 'etiquette_dpe'
data = data.dropna(subset=[target])

In [5]:
ls_variables_explicatives = [
    'conso_5_usages_par_m2_ep',
    'emission_ges_5_usages_par_m2',
    'conso_5_usages_par_m2_ef',
    'conso_chauffage_ep',
    'emission_ges_chauffage',
    'cout_chauffage',
    'besoin_ecs',
    'surface_habitable_logement',
    'cout_total_5_usages',
    'conso_5_usages_ef_energie_n1',
    'conso_ecs_ep',
    'conso_eclairage_ef',
    'type_energie_principale_chauffage',
    'conso_chauffage_ef',
    'annee_construction'
]


In [6]:
# Définir les données d'entrée et la variable cible
X = data[ls_variables_explicatives]
y = data[target]


In [7]:
X.head(3)

,conso_5_usages_par_m2_ep,emission_ges_5_usages_par_m2,conso_5_usages_par_m2_ef,conso_chauffage_ep,emission_ges_chauffage,cout_chauffage,besoin_ecs,surface_habitable_logement,cout_total_5_usages,conso_5_usages_ef_energie_n1,conso_ecs_ep,conso_eclairage_ef,type_energie_principale_chauffage,conso_chauffage_ef,annee_construction
0,282.0,8.0,276.0,31705.6,951.2,1732.4,1499.7,126.6,2058.3,28521.3,2803.0,235.4,Bois – Granulés (pellets) ou briquettes,31705.6,1948
1,282.0,8.0,276.0,31705.6,951.2,1732.4,1499.7,126.6,2058.3,28521.3,2803.0,235.4,Bois – Granulés (pellets) ou briquettes,31705.6,1948
2,168.0,5.0,150.0,20725.5,638.3,1287.8,1657.2,157.0,1615.2,18911.5,4511.9,292.0,Bois – Bûches,18579.2,1948


In [8]:
y.head(3)

0    E
1    E
2    C
Name: etiquette_dpe, dtype: object

In [9]:
# Séparer les colonnes numériques et catégorielles
numeric_features = data[ls_variables_explicatives].select_dtypes(include=[np.number]).columns
categorical_features = data[ls_variables_explicatives].select_dtypes(exclude=[np.number]).columns

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Création des transformations pour chaque type de données
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),      # Impute les valeurs manquantes par la mediane
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute les valeurs manquantes par la valeur la plus fréquente
    ('ordinal', OrdinalEncoder())  # Encodage des variables catégorielles
])


# Application des transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Création du pipeline complet
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, min_samples_split=5, random_state=0, verbose=1))
])

In [12]:
#Entraînement du modèle
pipeline.fit(X_train, y_train)


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    5.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   11.5s finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['conso_5_usages_par_m2_ep', 'emission_ges_5_usages_par_m2',
       'conso_5_usages_par_m2_ef', 'conso_chauffage_ep',
       'emission_ges_chauffage', 'cout_chauffage', 'besoin_ecs',
       'surface_habitable_logement', 'cout_total_5_usages',
       'conso_5_usages_ef_energie_n1', 'conso_ecs_ep', 'conso_eclairage_ef',
       'conso_chauffage_ef', 'annee_construction'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  Index(['type_energie_principale_chauffage'], dtype='object'))])),
                ('classifier',
                 RandomForestClassifier(min_samples_split=5, random_state=0,
                                        verbose=1))])

In [14]:
# 5️⃣ Sauvegarder le pipeline dans un fichier .pkl
joblib.dump(pipeline, '../Modèle/pipeline.pkl')

['../Modèle/pipeline.pkl']

In [15]:
rf_y_pred = pipeline.predict(X_test)
rf_y_pred

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


array(['D', 'G', 'E', ..., 'E', 'E', 'G'], dtype=object)

In [16]:
#Évaluation du modèle
print("Matrice de confusion :\n", confusion_matrix(y_test, rf_y_pred))
print("Rapport de classification :\n", classification_report(y_test, rf_y_pred))

Matrice de confusion :
 [[  600     0     0     0     0     0]
 [    0  1836     0     0     0     0]
 [    0     0 12097     0     0     0]
 [    0     0     0 15486     0     0]
 [    0     0     0     0  9670     0]
 [    0     0     0     0     0 11652]]
Rapport de classification :
               precision    recall  f1-score   support

           B       1.00      1.00      1.00       600
           C       1.00      1.00      1.00      1836
           D       1.00      1.00      1.00     12097
           E       1.00      1.00      1.00     15486
           F       1.00      1.00      1.00      9670
           G       1.00      1.00      1.00     11652

    accuracy                           1.00     51341
   macro avg       1.00      1.00      1.00     51341
weighted avg       1.00      1.00      1.00     51341



In [17]:
# Chargement et utilisation du pipeline sauvegardé
pipeline_loaded = joblib.load('../Modèle/pipeline.pkl')

In [18]:
y_pred_loaded = pipeline_loaded.predict(X_test)
print("Évaluation du pipeline chargé :")
print("Matrice de confusion :\n", confusion_matrix(y_test, y_pred_loaded))
print("Rapport de classification :\n", classification_report(y_test, y_pred_loaded))

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


Évaluation du pipeline chargé :
Matrice de confusion :
 [[  600     0     0     0     0     0]
 [    0  1836     0     0     0     0]
 [    0     0 12097     0     0     0]
 [    0     0     0 15486     0     0]
 [    0     0     0     0  9670     0]
 [    0     0     0     0     0 11652]]
Rapport de classification :
               precision    recall  f1-score   support

           B       1.00      1.00      1.00       600
           C       1.00      1.00      1.00      1836
           D       1.00      1.00      1.00     12097
           E       1.00      1.00      1.00     15486
           F       1.00      1.00      1.00      9670
           G       1.00      1.00      1.00     11652

    accuracy                           1.00     51341
   macro avg       1.00      1.00      1.00     51341
weighted avg       1.00      1.00      1.00     51341

